In [11]:
#@author : https://github.com/nitish11  
#@decription : finding relationship betwen two sentences 
#@date : 31st June, 2017'

## Objective : 

* Text classification for given two columns of data *train_data[[1,2]]*.
* No. of classes = 5 *[' ForwardEntailment', ' Independent', ' Equivalence', ' ReverseEntailment', ' OtherRelated']*

* Other columns contain extra  information about the two segments of sentences

## Approach 

### Using classical method of classification

* Preprcoessing of columns of the given data
    * Extract the numeral information from the data given as extra columns
    * Store only these columns as training data
* Use the extracted information from training data and perform classical methods (Random Forest, XGboost) for classification. 


### Deep Learning Solution 

* Use only starting 2 columns (text_data) and do pre-processing.
* Use word2vec or Glove for representing text as vectors.
* Use the vectors and do classiication using differen models as LSTM.

### Loading the data and preprocessing 

In [1]:
#import modules
import pandas as pd

('-- keys : ', Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17], dtype='int64'))
('--- :',                  1                     2
0   used to treat              to treat 
1    education is    education programs 
2    are reviewed      are under review )
('--- :', array([' ForwardEntailment', ' Independent', ' Equivalence',
       ' ReverseEntailment', ' OtherRelated'], dtype=object))


In [9]:
#Loading training data
train_data = pd.read_csv("ppdb.train.csv", header=None)

print("-- keys : ",train_data.keys())
print("---text_data:",train_data[[1,2]].head(3))
print("--- Results :",train_data[17].unique())
print("--- #of rows:",train_data.size)

('-- keys : ', Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17], dtype='int64'))
('---text_data:',                  1                     2
0   used to treat              to treat 
1    education is    education programs 
2    are reviewed      are under review )
('--- Results :', array([' ForwardEntailment', ' Independent', ' Equivalence',
       ' ReverseEntailment', ' OtherRelated'], dtype=object))
('--- #of rows:', 18000)


In [5]:
# Preprocessing of result column
train_data.loc[train_data[17] == ' ForwardEntailment', 17] = 0
train_data.loc[train_data[17] == ' Independent', 17] = 1
train_data.loc[train_data[17] == ' Equivalence', 17] = 2
train_data.loc[train_data[17] == ' ReverseEntailment', 17] = 3
train_data.loc[train_data[17] == ' OtherRelated', 17] = 4
train_data[17].unique()


array([0, 1, 2, 3, 4], dtype=object)